<a href="https://colab.research.google.com/github/jmse0/myrepo/blob/master/Bert_sjm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/silverkhan/krDataScience/master/ml02/CRSE_df.pkl

--2020-08-06 05:36:40--  https://raw.githubusercontent.com/silverkhan/krDataScience/master/ml02/CRSE_df.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 834462 (815K) [application/octet-stream]
Saving to: ‘CRSE_df.pkl’

CRSE_df.pkl         100%[===================>] 814.90K  --.-KB/s    in 0.07s   

2020-08-06 05:36:41 (11.0 MB/s) - ‘CRSE_df.pkl’ saved [834462/834462]



In [1]:
# !pip install bert
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 3.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=e26a93019aa7f6a18c7cb5c053832b18efa5cad4206fe19c442e1ed8e4effa8d
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=263ef7eae77272af6c6d51668175f0982c2c3a6d8ae7d8b2ab6eb0e3fe011109
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=b3c0b8a67630099f60702014f6e1c59ac4fec9bf95ffecdcfa12350daea499c2
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, Activation, LSTM, SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bert
from tqdm import tqdm
from tensorflow.keras import backend as K
import tensorflow as tf
import tensorflow_hub as hub
from bert import bert_tokenization
import time
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.3.0
Hub version:  0.8.0


In [4]:
#컬럼명
topicCol_char = "CRSE_CL"
labelCol_char = "STD_CD"
padIntCol_char = "PADDED_NM"
crtYM_char = "CRT_YM"
# 변수
testSplit_char = "2020.05"
# Path 정의
curDataPath_char = "https://raw.githubusercontent.com/silverkhan/krDataScience/master/ml02/CRSE_df_0803.pkl"
# pastDataPath_char = "https://raw.githubusercontent.com/silverkhan/krDataScience/master/ml02/PAST_df.pkl"
curRaw_df = pd.read_pickle(curDataPath_char)
# pastRaw_df = pd.read_pickle(pastDataPath_char)
# Testset 정의
rawTest_df = curRaw_df.query(crtYM_char + " >= " + testSplit_char).copy()
rawTrain_df = curRaw_df.query(crtYM_char + " < " + testSplit_char).copy()

In [5]:
print(curRaw_df.shape)
print(rawTest_df.shape)
print(rawTrain_df.shape)

(3725, 8)
(106, 8)
(3619, 8)


In [ ]:
# checkBtw = pd.merge(check_raw, check_train, on = 'CRT_YN')
# checkBtw["check"] = checkBtw['rawFrq'] == checkBtw["trainFrq"]

In [ ]:
# checkBtw

,CRT_YN,rawFrq,trainFrq,check
0,2017.01,75,75,True
1,2017.02,163,163,True
2,2017.03,113,113,True
3,2017.04,79,79,True
4,2017.05,63,63,True
5,2017.06,58,58,True
6,2017.07,27,27,True
7,2017.08,62,62,True
8,2017.09,56,56,True
9,2017.10,54,54,True


In [6]:
display(curRaw_df)

,CRSE_CD,CRSE_NM,CRSE_BERT,CRSE_CL,PADDED_NM,CRT_YM,STD_CAT,STD_CD
0,AAAA009,(중앙회)5급신규직원입문과정(직무),신규직원입문 직무,"[신규, 직원, 입문, 직무]","[77, 69, 95, 148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2018.12,직원일반,4
1,AABA096,하나로유통 세무회계 실무,하나로유통 세무회계 실무,"[하나, 로, 유통, 세무, 회계, 실무]","[193, 28, 43, 92, 58, 4, 0, 0, 0, 0, 0, 0, 0, ...",2017.04,회계업무,7
2,AABA097,협동조합전문가역량강화과정,협동조합전문가역량강화,"[협동조합, 전문가, 역량, 강화]","[339, 19, 89, 106, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2018.03,직원일반,4
3,AAPA010,농축협 순회검사역 과정,농축협 순회검사역 과정,"[농, 축협, 순회, 검사역, 과정]","[8, 26, 2481, 1385, 47, 0, 0, 0, 0, 0, 0, 0, 0...",2019.03,직원일반,4
4,AD3A001,(생명) 직무전문가 육성체계 및 교육프로그램 사례발표회,직무전문가 육성체계 및 교육프로그램 사례발표회,"[직무, 전문가, 육성, 체계, 및, 교육, 프로그램, 사례, 발표회]","[148, 19, 713, 764, 14, 9, 737, 62, 890, 0, 0,...",2018.04,강사양성과정,41
...,...,...,...,...,...,...,...,...
3720,LSDM001,상호금융 리더를 위한 DT-AI 인사이트,상호금융 리더를 위한 DTAI 인사이트,"[상호, 금융, 리더, 를, 위한, DTAI, 인, 사이트]","[184, 7, 70, 12, 13, 3442, 80, 846, 0, 0, 0, 0...",2019.08,산업동향,1
3721,LSPM001,빅데이터를 활용한 CRM고객관리,빅데이터를 한 CRM고객관리,"[빅, 데이터, 를, 한, CRM, 고객, 관리]","[125, 59, 12, 22, 377, 41, 11, 0, 0, 0, 0, 0, ...",2019.05,영업마케팅,2
3722,LTBK001,재무리스크 측정과 관리실무,재무리스크 측정과 관리실무,"[재무, 리스크, 측정, 과, 관리, 실무]","[86, 143, 1141, 21, 11, 4, 0, 0, 0, 0, 0, 0, 0...",2019.10,금융리스크,22
3723,LTDK001,북한비즈니스,북한비즈니스,"[북한, 비즈니스]","[2979, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2020.04,금융교육과정,10


In [ ]:
#7/30 삭제 labelCol_Num_char = "['1',  '2',  '3',  '4',  '5',  '6',  '7',  '8',  '9',  '10',  '11',  '12',  '13',  '14',  '15',  '16',  '17',  '18',  '19',  '20',  '21',  '22',  '23',  '24',  '25',  '26',  '27',  '28',  '29',  '30',  '31',  '32',  '33',  '34',  '35',  '36',  '37',  '38',  '39',  '40',  '41',  '42',  '43',  '44',  '45',  '46',  '47',  '48',  '49',  '50',  '51',  '52', '53']"
# labelCol_Num_char = "['1',  '2',  '3',  '4',  '5']"
#7/30 삭제 f_rawTest_df = rawTest_df.query(labelCol_char + " in " + labelCol_Num_char)
#7/30 삭제 f_rawTrain_df = rawTrain_df.query(labelCol_char + " in " + labelCol_Num_char)

In [ ]:
# x = np.unique(rawTrain_df[labelCol_char], return_counts=True)
# check_ori = pd.DataFrame(data = {labelCol_char:x[0], 'oriFrq':x[1]})
#7/30 삭제 x = np.unique(f_rawTrain_df[labelCol_char], return_counts=True)
#7/30 삭제 check_f = pd.DataFrame(data = {labelCol_char:x[0], 'fFrq':x[1]})

In [ ]:
# checkBtw

,CRT_YN,rawFrq,trainFrq,check
0,2017.01,75,75,True
1,2017.02,163,163,True
2,2017.03,113,113,True
3,2017.04,79,79,True
4,2017.05,63,63,True
5,2017.06,58,58,True
6,2017.07,27,27,True
7,2017.08,62,62,True
8,2017.09,56,56,True
9,2017.10,54,54,True


In [ ]:
# rawTrain_df[rawTrain_df.STD_CD == '52']

,CRSE_CD,CRSE_NM,CRSE_BERT,CRSE_CL,PADDED_NM,CRT_YM,STD_CAT,STD_CD
441,BA3K001,기업금융RM단기과정,기업금융RM단기,"[기업, 금융, RM, 단기]","[31, 7, 559, 493, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2019.10,RM교육,52
453,BAPA047,영업본부 현장지원단 RM 기본교육(은행),영업본부 현장지원단 RM 기본교육,"[영업, 본부, 현장, 지원, 단, RM, 기본, 교육]","[102, 1518, 221, 424, 2607, 559, 51, 9, 0, 0, ...",2017.02,RM교육,52
460,BAPA054,기업금융RM 단기과정(은행),기업금융RM 단기,"[기업, 금융, RM, 단기]","[31, 7, 559, 493, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2018.10,RM교육,52
696,BIBA576,영업본부 RM심화과정(은행),영업본부 RM심화,"[영업, 본부, RM, 심화]","[102, 1518, 559, 183, 0, 0, 0, 0, 0, 0, 0, 0, ...",2017.08,RM교육,52
707,BIBA590,기업금융 RM 기초(은행),기업금융 RM 기초,"[기업, 금융, RM, 기초]","[31, 7, 559, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2018.01,RM교육,52
715,BIBA598,농협은행 경기영업본부 RM 실습,농협은행 경기영업본부 RM 실습,"[농협, 은행, 경기, 영업, 본부, RM, 실습]","[34, 154, 1428, 102, 1518, 559, 292, 0, 0, 0, ...",2018.01,RM교육,52
731,BIPA014,영업본부 RM기본과정,영업본부 RM기본,"[영업, 본부, RM, 기본]","[102, 1518, 559, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2018.01,RM교육,52
927,BU3K014,기업금융RM고급과정,기업금융RM고급,"[기업, 금융, RM, 고급]","[31, 7, 559, 202, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2019.10,RM교육,52
1254,FABA163,현장지원단 RM 기본교육(은행),현장지원단 RM 기본교육,"[현, 장지, 원단, RM, 기본, 교육]","[938, 5174, 2518, 559, 51, 9, 0, 0, 0, 0, 0, 0...",2017.02,RM교육,52


In [ ]:
# print(check_ori.shape)
# print(check_f.shape)

(52, 2)


In [ ]:
# ['1',  '2',  '3',  '4',  '5',  '6',  '7',  '8',  '9',  '10',  '11',  '12',  '13',  '14',  '15',  '16',  '17',  '18',  '19',  '20',  '21',  '22',  '23',  '24',  '25',  '26',  '27',  '28',  '29',  '30',  '31',  '32',  '33',  '34',  '35',  '36',  '37',  '38',  '39',  '40',  '41',  '42',  '43',  '44',  '45',  '46',  '47',  '48',  '49',  '50',  '51',  '52']"

In [7]:
class LoadingData():

    def __init__(self):
      self.train_data_frame = rawTrain_df.copy()

      # self.curRaw_category = pd.read_csv("/content/drive/My Drive/Colab Notebooks/labelMst.txt", sep='\t')

      self.curRaw_category = curRaw_df[['STD_CAT', 'STD_CD']].drop_duplicates(['STD_CAT'])
      self.cat_to_intent = self.curRaw_category.set_index('STD_CAT').T.to_dict('records')[0]

      # curRaw_category = curRaw_df[['STD_CAT', 'STD_CD']].drop_duplicates(['STD_CAT'])
      # self.cat_to_intent = curRaw_category.set_index('STD_CAT').T.to_dict('records')[0]
      # self.cat_to_intent = {k : int(v) for k, v in self.cat_to_intent.items()}
      self.intent_to_cat = {self.cat_to_intent[x]:x for x in self.cat_to_intent}

      self.train_data_frame = self.train_data_frame.rename(columns={'CRSE_BERT' : 'id', 'CRSE_NM' : 'query', 'STD_CAT' : 'intent','STD_CD' : 'category'})
      self.train_data_frame = self.train_data_frame[['id','query','intent','category']]
      self.train_data_frame['category'] = pd.to_numeric(self.train_data_frame['category'])

      self.validation_data_frame = rawTest_df.copy()
      self.validation_data_frame = self.validation_data_frame.rename(columns={'CRSE_BERT' : 'id', 'CRSE_NM' : 'query', 'STD_CAT' : 'intent','STD_CD' : 'category'})
      self.validation_data_frame = self.validation_data_frame[['id','query','intent','category']]
      self.validation_data_frame['category'] = pd.to_numeric(self.validation_data_frame['category'])
      # 2, 3, 8

      print(len(self.train_data_frame), len(self.validation_data_frame))
      print(self.train_data_frame.index.sort_values(), self.validation_data_frame.index.sort_values(), sep='\n')

In [ ]:
# curRaw_category = pd.read_csv("/content/drive/My Drive/Colab Notebooks/labelMst.txt", sep='\t')
# len(curRaw_category)

NameError: ignored

In [8]:
load_data_obj = LoadingData()

3619 106
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3715, 3716, 3717, 3718, 3719, 3720, 3721, 3722, 3723, 3724],
           dtype='int64', length=3619)
Int64Index([ 607,  732,  733,  855,  856,  857,  858,  859,  860,  861,
            ...
            3630, 3631, 3632, 3633, 3634, 3635, 3636, 3673, 3678, 3679],
           dtype='int64', length=106)


In [9]:
len(load_data_obj.cat_to_intent)

53

In [10]:
print("전체 데이터 수 : ", len(load_data_obj.train_data_frame))
load_data_obj.train_data_frame['category'].value_counts().sort_index()

전체 데이터 수 :  3619


1     259
2     231
3     222
4     171
5     141
6     144
7     126
8     117
9     115
10    118
11    114
12    114
13    107
14    108
15    102
16     91
17     90
18     89
19     77
20     79
21     75
22     72
23     65
24     60
25     57
26     58
27     43
28     38
29     36
30     33
31     35
32     34
33     32
34     32
35     25
36     27
37     27
38     25
39     24
40     24
41     21
42     19
43     18
44     18
45     18
46     15
47     14
48     13
49     11
50     10
51      9
52      9
53      7
Name: category, dtype: int64

In [11]:
print("전체 데이터 수 : ", len(load_data_obj.validation_data_frame))
load_data_obj.validation_data_frame['category'].value_counts().sort_index()

전체 데이터 수 :  106


1     17
2      7
3      1
4      2
5     10
6      1
7      4
8      6
9      6
10     2
11     3
13     5
14     1
15     3
16     3
17     3
18     1
19     6
22     1
23     5
24     1
25     2
29     1
30     3
33     1
34     1
35     5
36     1
41     1
43     1
47     1
49     1
Name: category, dtype: int64

In [12]:
class BertModel(object):
    
    def __init__(self):
        
        self.max_len = 128
        # bert_path = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
        bert_path = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2"
        FullTokenizer=bert.bert_tokenization.FullTokenizer
        
        self.bert_module = hub.KerasLayer(bert_path,trainable=True)

        self.vocab_file = self.bert_module.resolved_object.vocab_file.asset_path.numpy()

        # self.do_lower_case = self.bert_module.resolved_object.do_lower_case.numpy()
        self.do_lower_case = False
        self.tokenizer = FullTokenizer(self.vocab_file,self.do_lower_case)
        
    def get_masks(self,tokens, max_seq_length):
        return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

    def get_segments(self,tokens, max_seq_length):
        """Segments: 0 for the first sequence, 1 for the second"""
        segments = []
        current_segment_id = 0
        for token in tokens:
            segments.append(current_segment_id)
            if token == "[SEP]":
                current_segment_id = 1
        return segments + [0] * (max_seq_length - len(tokens))
    
    def get_ids(self,tokens, tokenizer, max_seq_length):
        """Token ids from Tokenizer vocab"""
        token_ids = tokenizer.convert_tokens_to_ids(tokens,)
        input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
        return input_ids
    def create_single_input(self,sentence,maxlen):

        stokens = self.tokenizer.tokenize(sentence)

        stokens = stokens[:maxlen]

        stokens = ["[CLS]"] + stokens + ["[SEP]"]

        ids = self.get_ids(stokens, self.tokenizer, self.max_len)
        masks = self.get_masks(stokens, self.max_len)
        segments = self.get_segments(stokens, self.max_len)
        input_data = [ids,masks,segments]

        return input_data

    def create_input_array(self,sentences):
        
        input_ids, input_masks, input_segments = [], [], []

        for sentence in tqdm(sentences,position=0, leave=True):
            ids,masks,segments=self.create_single_input(sentence,self.max_len-2)

            input_ids.append(ids)
            input_masks.append(masks)
            input_segments.append(segments)
            
        input_array = [np.asarray(input_ids, dtype=np.int32), 
                np.asarray(input_masks, dtype=np.int32), 
                np.asarray(input_segments, dtype=np.int32)]
        return input_array

In [13]:
class PreprocessingBertData():
    def prepare_data_x(self,train_sentences):
      x = bert_model_obj.create_input_array(train_sentences)
      return x
    
    def prepare_data_y(self,train_labels):
        y = list()
        for item in train_labels:
            label = item
            y.append(label)
        y = np.array(y)
        return y

In [14]:
bert_model_obj = BertModel()

In [15]:
def preprocessInputLabels_func(data_frame):
  sentences = data_frame["query"].tolist()
  labels = data_frame["category"].tolist()

  preprocess_bert_data_obj = PreprocessingBertData()
  x = preprocess_bert_data_obj.prepare_data_x(sentences)
  y = preprocess_bert_data_obj.prepare_data_y(labels)

  input_ids, input_masks, segment_ids = x
  labels = y

  return [input_ids, input_masks, segment_ids] , labels

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [21]:
class DesignModel():
    def __init__(self):
        self.model = None        
        self.train_data, self.train_labels = preprocessInputLabels_func(load_data_obj.train_data_frame)
        self.validation_data, self.validation_labels = preprocessInputLabels_func(load_data_obj.validation_data_frame)
        
    def bert_model(self,max_seq_length): 

        in_id = Input(shape=(max_seq_length,), dtype=tf.int32, name="input_ids")
        in_mask = Input(shape=(max_seq_length,), dtype=tf.int32, name="input_masks")
        in_segment = Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")
        bert_inputs = [in_id, in_mask, in_segment]
        bert_pooled_output, bert_sequence_output = bert_model_obj.bert_module(bert_inputs)
        bert_output = tf.keras.layers.GlobalAveragePooling1D()(bert_sequence_output)
        # bert_output = tf.keras.layers.Dropout(0.5)(bert_output)
        # bert_output = tf.keras.layers.Dropout(0.2)(bert_output) # ori
        
        # bert_outputs = tf.keras.layers.Dense(len(load_data_obj.cat_to_intent), /activation="softmax", name="dense_output")(bert_output)
        
        # bert_output = tf.keras.layers.Dense(1, activation="softmax")(bert_sequence_output)
        # bert_outputs = tf.keras.layers.Dense(1, activation="softmax")(bert_output)
        # bert_outputs = tf.keras.layers.Dense(len(load_data_obj.cat_to_intent), activation="softmax", name="dense_output")(bert_output)

        bert_output = tf.keras.layers.Dense(128, activation="relu", name="dense_hidden_1st")(bert_output)
        bert_output = tf.keras.layers.Dropout(0.5)(bert_output)
        bert_outputs = tf.keras.layers.Dense(len(load_data_obj.cat_to_intent)+1, activation="softmax", name="dense_output")(bert_output)

        self.model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert_outputs)
        
        self.model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                           metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])
        '''
        self.model.compile(optimizer=tf.keras.optimizers.Adam(5e-5),
                           loss = 'binary_crossentropy',
                           metrics=['accuracy'])
        '''
        # Batch size: 16, 32 Learning rage (Adam): 5e-5, 3e-5, 2e-5
        self.model.summary()
    
    def model_train(self,batch_size,num_epoch):
        print("Fitting to model")
        # class_weights = class_weight.compute_class_weight('balanced', np.unique(self.train_labels), self.train_labels)
        # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
        checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/model_200811.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
        self.model.fit(self.train_data,self.train_labels,epochs=num_epoch,batch_size=batch_size, validation_data = (self.validation_data, self.validation_labels), callbacks=[checkpointer], verbose=1, shuffle=True)
        print("Model Training complete.")
# , callbacks=[checkpointer], verbose=1, shuffle=True
    def save_model(self,model,model_name):    
        self.model.save(model_name+".h5")
        print("Model saved to Model folder.")

In [22]:
# start = time.time()
model_obj = DesignModel()
model_obj.bert_model(bert_model_obj.max_len)
# model_obj.model_train(16,30)
model_obj.model_train(32, 30)
# 1/2 loss: 3.4101 - acc: 0.1735 - val_loss: 2.3932 - val_acc: 0.4340
# 2/2 loss: 2.0580 - acc: 0.5162 - val_loss: 1.7275 - val_acc: 0.6038

100%|██████████| 106/106 [00:00<00:00, 8072.56it/s]


Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 177853441   input_ids[0][0]                  
                                                                 input_masks[0][0]     

In [ ]:
..# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# model_obj.model_train(16,10)

Fitting to model
Epoch 1/10
227/227 [==============================] - ETA: 0s - loss: 0.0231 - acc: 0.9942
Epoch 00001: val_loss improved from inf to 1.27346, saving model to /content/drive/My Drive/bert_model/model_best.h5
227/227 [==============================] - 197s 868ms/step - loss: 0.0231 - acc: 0.9942 - val_loss: 1.2735 - val_acc: 0.8019
Epoch 2/10
227/227 [==============================] - ETA: 0s - loss: 0.0138 - acc: 0.9970
Epoch 00002: val_loss did not improve from 1.27346
227/227 [==============================] - 197s 868ms/step - loss: 0.0138 - acc: 0.9970 - val_loss: 1.3613 - val_acc: 0.7925
Epoch 3/10
227/227 [==============================] - ETA: 0s - loss: 0.0181 - acc: 0.9947
Epoch 00003: val_loss improved from 1.27346 to 1.21492, saving model to /content/drive/My Drive/bert_model/model_best.h5
227/227 [==============================] - 197s 868ms/step - loss: 0.0181 - acc: 0.9947 - val_loss: 1.2149 - val_acc: 0.8208
Epoch 4/10
227/227 [==========================

In [ ]:
model_obj.save_model(model_obj.model,"0731_accu_84")

Model saved to Model folder.


In [ ]:
model_obj.model.save('path_to_saved_model', save_format="tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: path_to_saved_model/assets


INFO:tensorflow:Assets written to: path_to_saved_model/assets


In [ ]:
model_obj.model.save("/content/gdrive/My Drive/Colab Notebooks/model_sjm_200731.h5")

In [ ]:
from tensorflow.keras.models import load_model

model_obj_path = '/content/gdrive/My Drive/Colab Notebooks/model_sjm_200803.h5'

new_model = load_model(model_obj_path, custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
class Evaluation():
    def get_accuracy(self,actuals, predictions):
        acc = accuracy_score(actuals, predictions)
        return acc

In [ ]:
class Prediction():
    def __init__(self):
        self.model = model_obj.model
        
    def predict_validation(self, data_frame):
        valid_sentences = data_frame["query"].tolist()
        valid_labels = data_frame["category"].tolist()

        preprocess_bert_data_obj = PreprocessingBertData()
        val_x = preprocess_bert_data_obj.prepare_data_x(valid_sentences)
        prediction_labels = list(self.model.predict(val_x).argmax(axis=-1))
        return valid_labels,prediction_labels
        
    
    def predict(self,query):
        query_seq = bert_model_obj.create_input_array([query])
        pred = self.model.predict(query_seq)
        pred = np.argmax(pred)
        result = load_data_obj.intent_to_cat[str(pred)]
        return result

In [ ]:
print(load_data_obj.cat_to_intent)

{'직원일반': '4', '회계업무': '7', '강사양성과정': '41', '프로그래밍일반': '13', '소프트웨어공학': '42', '데이터분석:일반': '8', '웹/앱개발': '25', 'OS일반': '23', '데이터분석:머신러닝딥러닝': '24', '데이터베이스/프로그래밍': '17', '정보보안': '12', '금융투자신탁': '15', '법률교육과정': '26', '기업금융': '29', '경영교육과정': '3', '여신업무': '6', '내부감사': '39', 'MBA과정': '33', '세무업무': '31', '유통교육과정': '20', '최고위과정': '32', '마인드컨드롤': '38', '금융교육과정': '10', '네트워크통신': '49', '커뮤니케이션스킬': '21', '영업마케팅': '2', '프로젝트관리': '36', '블록체인': '27', 'IT실무': '28', '문서작성활용': '9', '생산품질관리': '34', '인사관리': '46', '개인역량강화': '16', '구매업무': '51', '개인고객금융업무': '11', '사물인터넷(IoT)': '48', '산업동향': '1', '리더십역량강화': '18', '농축산업교육': '14', '인문학': '44', 'RM교육': '52', '보험업무': '5', '수신업무': '53', '금융리스크': '22', '외환업무': '35', '부동산관련과정': '19', '상품기획역량': '43', '언어교육': '37', '글로벌역량교육': '45', '고객상담': '47', '건설업무': '40', '소매관리': '30', '안전관리교육': '50'}


In [ ]:
pred_obj = Prediction()
#pred_obj.predict_validation()

In [ ]:
querylist = load_data_obj.validation_data_frame.values.tolist()

In [ ]:
querylist[0:10]

[['KBI 금융DT테스트', 'KBI 금융DT테스트', '산업동향', 1],
 ['여신심사', '(주말) 여신심사 (은행)', '여신업무', 6],
 ['토요 VIP 고객과의 소통을 위한 문화강좌', '토요 VIP 고객과의 소통을 위한 문화강좌', '개인고객금융업무', 11],
 ['직장인을 위한 실무 파워포인트 2016', '직장인을 위한 실무 파워포인트 2016(금융지주)', '문서작성활용', 9],
 ['어서와 데이터는 처음이지 기초 OF 기초 데이터 개념',
  '어서와 데이터는 처음이지 - 기초 of 기초 데이터 개념(금융지주)',
  '데이터분석:일반',
  8],
 ['어서와 데이터는 처음이지 미래 예측을 위한 기초 개념',
  '어서와 데이터는 처음이지 - 미래 예측을 위한 기초 개념(금융지주)',
  '데이터분석:일반',
  8],
 ['어서와 데이터는 처음이지 효과 검증을 위한 기초 개념',
  '어서와 데이터는 처음이지 - 효과 검증을 위한 기초 개념(금융지주)',
  '데이터분석:일반',
  8],
 ['산업혁명 시대의 비즈니스 혁명 플랫폼을 잡아라',
  '4차 산업혁명 시대의 비즈니스 혁명, 플랫폼을 잡아라(금융지주)',
  '산업동향',
  1],
 ['KIFRS Ⅱ', 'K-IFRS Ⅱ(금융지주)', '회계업무', 7],
 ['인공지능 AI 프로그래밍파이썬을 활용한 머신러닝 딥러닝 기초 다지기 PART2',
  '인공지능(AI) 프로그래밍_파이썬을 활용한 머신러닝, 딥러닝 기초 다지기 Part.2(금융지주)',
  '데이터분석:머신러닝딥러닝',
  24]]

In [ ]:
for query in querylist[0:50]:
  result = pred_obj.predict(query[1])
  # print("\n■Predicted Intent: "+str(result)+"\tActual Intent: "+(load_data_obj.cat_to_intent[query[2]]))
  # print(str(load_data_obj.intent_to_cat[str(query[3])]))
  print("■ Predicted Intent: "+str(result)+"\tActual Intent: "+str(load_data_obj.intent_to_cat[str(query[3])])+"\tquery: "+(query[1]), sep="\n\n")
  # print("■ Predicted Intent: "+str(result)+"\tActual Intent: "+ query[2]+"\tquery: "+(query[1]), sep="\n\n")

100%|██████████| 1/1 [00:00<00:00, 673.35it/s]

■ Predicted Intent: MBA과정	Actual Intent: 산업동향	query: KBI 금융DT테스트
■ Predicted Intent: 여신업무	Actual Intent: 여신업무	query: (주말) 여신심사 (은행)
■ Predicted Intent: 직원일반	Actual Intent: 개인고객금융업무	query: 토요 VIP 고객과의 소통을 위한 문화강좌
■ Predicted Intent: 문서작성활용	Actual Intent: 문서작성활용	query: 직장인을 위한 실무 파워포인트 2016(금융지주)
■ Predicted Intent: 데이터분석:일반	Actual Intent: 데이터분석:일반	query: 어서와 데이터는 처음이지 - 기초 of 기초 데이터 개념(금융지주)



100%|██████████| 1/1 [00:00<00:00, 655.56it/s]


■ Predicted Intent: 데이터분석:일반	Actual Intent: 데이터분석:일반	query: 어서와 데이터는 처음이지 - 미래 예측을 위한 기초 개념(금융지주)
■ Predicted Intent: 데이터분석:일반	Actual Intent: 데이터분석:일반	query: 어서와 데이터는 처음이지 - 효과 검증을 위한 기초 개념(금융지주)
■ Predicted Intent: 산업동향	Actual Intent: 산업동향	query: 4차 산업혁명 시대의 비즈니스 혁명, 플랫폼을 잡아라(금융지주)
■ Predicted Intent: 회계업무	Actual Intent: 회계업무	query: K-IFRS Ⅱ(금융지주)


100%|██████████| 1/1 [00:00<00:00, 3802.63it/s]

...

■ Predicted Intent: 데이터베이스/프로그래밍	Actual Intent: 데이터베이스/프로그래밍	query: Elasticsearch Engineer
■ Predicted Intent: 웹/앱개발	Actual Intent: 웹/앱개발	query: 업무에 바로 쓰는 HTML5와 CSS3
■ Predicted Intent: 프로그래밍일반	Actual Intent: 프로그래밍일반	query: Java Framework - Spring, iBatis, MyBatis 연동
■ Predicted Intent: 웹/앱개발	Actual Intent: 웹/앱개발	query: Swift Programming
■ Predicted Intent: 프로그래밍일반	Actual Intent: 프로그래밍일반	query: 파이썬 핵심


100%|██████████| 1/1 [00:00<00:00, 1471.17it/s]

■ Predicted Intent: 프로그래밍일반	Actual Intent: 프로그래밍일반	query: Spring Framework를 활용한 OpenAPI 서비스 개발
■ Predicted Intent: 데이터분석:일반	Actual Intent: 데이터분석:일반	query: [Big Data]R을 활용한 빅데이터 분석 및 활용실무
■ Predicted Intent: OS일반	Actual Intent: OS일반	query: AWS로 시작하는 클라우드 컴퓨팅(참고도서제공)[스마트러닝]
■ Predicted Intent: 데이터분석:일반	Actual Intent: 데이터분석:일반	query: 데이터분석준전문가(ADsP)_데이터의 이해(참고도서제공)[스마트러닝]
■ Predicted Intent: 개인고객금융업무	Actual Intent: 네트워크통신	query: 쉽게 배우는 시스코랜스위칭(참고도서제공)[스마트러닝]



100%|██████████| 1/1 [00:00<00:00, 525.34it/s]


■ Predicted Intent: OS일반	Actual Intent: OS일반	query: Perfect! 클라우드 데이터센터 핵심 기술 실무
■ Predicted Intent: 데이터베이스/프로그래밍	Actual Intent: 데이터베이스/프로그래밍	query: Oracle Database 12c: SQL Tuning Workshop & Case Study
■ Predicted Intent: 데이터베이스/프로그래밍	Actual Intent: 데이터베이스/프로그래밍	query: Oracle Database: Performance Management and Tuning


In [ ]:
rawTrain_df["train/test"] = "train"
rawTest_df["train/test"] = "test"

ytrain,predicted_ml_train = pred_obj.predict_validation(load_data_obj.train_data_frame)
ytest,predicted_ml_test = pred_obj.predict_validation(load_data_obj.validation_data_frame)

result_train = pd.concat([rawTrain_df.reset_index(), pd.Series(predicted_ml_train, name="Result")], axis=1)
result_test = pd.concat([rawTest_df.reset_index(), pd.Series(predicted_ml_test, name="Result")], axis=1)


100%|██████████| 106/106 [00:00<00:00, 9367.41it/s]


In [ ]:
curRaw_df_result = pd.merge(curRaw_df.reset_index(), pd.concat(
        [result_test[['index', 'train/test', 'Result']], result_train[['index', 'train/test', 'Result']]],
        ignore_index=True), on='index')
curRaw_df_result = curRaw_df_result.astype({'Result': 'str'})
curRaw_df_result["hitYN"] = curRaw_df_result[labelCol_char] == curRaw_df_result["Result"]
curRaw_category = curRaw_df[['STD_CAT', 'STD_CD']].drop_duplicates(['STD_CAT'])
cd_to_cat = curRaw_category.set_index('STD_CD').T.to_dict('records')[0]
# cd_to_cat = {int(k):v for k, v in cd_to_cat.items()}
curRaw_df_result['Result_CAT'] = curRaw_df_result['Result'].replace(cd_to_cat)
display(curRaw_df_result)

,index,CRSE_CD,CRSE_NM,CRSE_BERT,CRSE_CL,PADDED_NM,CRT_YM,STD_CAT,STD_CD,train/test,Result,hitYN,Result_CAT
0,0,AAAA009,(중앙회)5급신규직원입문과정(직무),신규직원입문 직무,"[신규, 직원, 입문, 직무]","[79, 71, 97, 153, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2018.12,직원일반,4,train,4,True,직원일반
1,1,AABA096,하나로유통 세무회계 실무,하나로유통 세무회계 실무,"[하나, 로, 유통, 세무, 회계, 실무]","[195, 29, 44, 90, 61, 4, 0, 0, 0, 0, 0, 0, 0, ...",2017.04,회계업무,7,train,7,True,회계업무
2,2,AABA097,협동조합전문가역량강화과정,협동조합전문가역량강화,"[협동조합, 전문가, 역량, 강화]","[344, 19, 91, 108, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2018.03,직원일반,4,train,4,True,직원일반
3,3,AAPA010,농축협 순회검사역 과정,농축협 순회검사역 과정,"[농, 축협, 순회, 검사역, 과정]","[8, 26, 2475, 1390, 50, 0, 0, 0, 0, 0, 0, 0, 0...",2019.03,직원일반,4,train,4,True,직원일반
4,4,AD3A001,(생명) 직무전문가 육성체계 및 교육프로그램 사례발표회,직무전문가 육성체계 및 교육프로그램 사례발표회,"[직무, 전문가, 육성, 체계, 및, 교육, 프로그램, 사례, 발표회]","[153, 19, 715, 762, 15, 9, 738, 65, 891, 0, 0,...",2018.04,강사양성과정,41,train,41,True,강사양성과정
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,3720,LSDM001,상호금융 리더를 위한 DT-AI 인사이트,상호금융 리더를 위한 DTAI 인사이트,"[상호, 금융, 리더, 를, 위한, DTAI, 인, 사이트]","[186, 7, 72, 12, 13, 3429, 83, 847, 0, 0, 0, 0...",2019.08,산업동향,1,train,1,True,산업동향
3721,3721,LSPM001,빅데이터를 활용한 CRM고객관리,빅데이터를 활용한 CRM고객관리,"[빅, 데이터, 를, 활용, 한, CRM, 고객, 관리]","[128, 63, 12, 28, 22, 379, 42, 11, 0, 0, 0, 0,...",2019.05,영업마케팅,2,train,2,True,영업마케팅
3722,3722,LTBK001,재무리스크 측정과 관리실무,재무리스크 측정과 관리실무,"[재무, 리스크, 측정, 과, 관리, 실무]","[88, 147, 1145, 21, 11, 4, 0, 0, 0, 0, 0, 0, 0...",2019.10,금융리스크,22,train,22,True,금융리스크
3723,3723,LTDK001,북한비즈니스,북한비즈니스,"[북한, 비즈니스]","[2968, 64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2020.04,금융교육과정,10,train,10,True,금융교육과정


In [ ]:
#파일저장
curRaw_df_result.to_csv("curRaw_df_result_0803.csv", index = False, encoding="UTF-8")

In [ ]:
eval_obj = Evaluation()
ytest,ypred = pred_obj.predict_validation(load_data_obj.validation_data_frame)
acc = eval_obj.get_accuracy(ytest,ypred)
print("Auc: {:.2%}".format(acc))

100%|██████████| 106/106 [00:00<00:00, 5311.28it/s]


Auc: 83.02%


In [ ]:
tensorflowjs.converters.convert_tf_saved_model(model_obj.model, '/content/drive/My Drive/bert')

TypeError: ignored

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

hub_url = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
embed = hub.KerasLayer(hub_url)
embeddings = embed(["A long sentence.", "single-word", "http://example.com"])
print(embeddings.shape, embeddings.dtype)

(3, 128) <dtype: 'float32'>


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# save model structure to JSON (no weights)
model_json = model_obj.model.to_json()
with open("model_to_json.json", "w") as json_file:
    json_file.write(model_json)
# saving the model weight separately
model_obj.model.save_weights("/content/drive/My Drive/bert/j_model.h5")

In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 92kB 4.6MB/s 
     |████████████████████████████████| 144.4MB 97kB/s 
     |████████████████████████████████| 256kB 43.3MB/s 
     |████████████████████████████████| 921kB 43.0MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=b1a5ccd01d2bdb83374f0076fce5a6f7680c3af46b98411f4f5e27eb09068d54
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: Pygments 2.1.3
    Uninstalling Pygments-2.1.3:
      Successfully uninstalled Pygments-2.1.3
  Found existing installation: tensorflow-hub 0.

In [ ]:
import tensorflowjs

In [ ]:
!mkdir model_obj.model

In [ ]:
!tensorflowjs_converter --input_format keras bert.h5 model/

In [ ]:
!tensorflowjs_converter --input_format keras --output_format tfjs_layers_model bert.json model/

Traceback (most recent call last):
  File "/usr/local/bin/tensorflowjs_converter", line 8, in <module>
    sys.exit(pip_main())
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/converter.py", line 735, in pip_main
    main([' '.join(sys.argv[1:])])
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/converter.py", line 739, in main
    convert(argv[0].split(' '))
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/converter.py", line 654, in convert
    weight_shard_size_bytes=weight_shard_size_bytes)
  File "/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/converter.py", line 74, in dispatch_keras_h5_to_tfjs_layers_model_conversion
    raise ValueError('Nonexistent path to HDF5 file: %s' % h5_path)
ValueError: Nonexistent path to HDF5 file: bert.json
